In [1]:
pip install -q openai python-dotenv langsmith

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: C:\Users\saraw\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [8]:
from openai import OpenAI
from langsmith import wrappers, traceable
import base64

client = wrappers.wrap_openai(OpenAI())

with open("vasculitis-101.jpg", "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

In [12]:
@traceable
def pipeline(image_base64):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a highly experienced dermatology assistant. "
                    "Given an image of a skin condition, your task is to identify the most likely diagnosis, "
                    "explain the reasoning briefly, and if applicable, suggest differential diagnoses."
                ),
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Classify the skin condition based on the image?",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_base64}",
                        },
                    },
                ],
            }
        ],
        max_tokens=500,
    )
    return response

response = pipeline(base64_image)

In [14]:
from IPython.display import Markdown

display(Markdown(response.choices[0].message.content))

This image shows ulcerated, crusted, and violaceous nodules and plaques, which is characteristic of cutaneous leishmaniasis. 

**Reasoning:**

1. **Ulcers:** The image shows open sores, which are typical of leishmaniasis.
2. **Crusting and Nodules:** The presence of crusted and nodular lesions aligns with the cutaneous form of this parasitic infection.
3. **Distribution:** The lesions are localized, which is common in cutaneous leishmaniasis.

**Differential Diagnoses:**
- **Pyoderma gangrenosum:** Usually presents with painful ulcers with a violaceous border.
- **Infections:** Other infectious processes such as ecthyma or sporotrichosis could present similarly but have different clinical histories and courses.
- **Vasculitis:** Can cause ulcerative lesions but typically has systemic involvement.

A clinical history, along with a biopsy or microbiological test, would aid in confirming the diagnosis.